In [1]:
import math
import pickle
import requests 
import json
import string
from pprint import pprint
from collections import Counter, defaultdict

from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
from nltk.corpus import stopwords
#from nameparser.parser import HumanName

In [3]:
topic = "politics"

In [6]:
swords = stopwords.words('english')
swords.append('\'s')
swords.append('\'t')
for s in swords:
    print s,

i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now 's 't


In [203]:
with open("guardian_api.txt", 'r') as f:
    api_key = f.next().strip()

In [204]:
def get_content(pg):
    api_url = 'http://content.guardianapis.com/search'
    payload = {
        'api-key':              api_key,
        'page-size':            50,
        'page':                 pg,
        #'q':                    'art OR music OR photography OR dance',
        #'q':                    'science OR astronomy OR space OR medicine',
        #'q':                    'sports OR soccer OR football OR game',
        'q':                    'party OR policy OR democrat OR republican', 
        #'order-by':             'newest',
        #'show-editors-picks':   'true',
        #'show-elements':        'image',
        'show-blocks':          'body',
        'show-fields':          'byline, wordcount',
        #'show-tags':            'all',
        #'show-references':      'author',
        'format':               'json'
    }
    response = requests.get(api_url, params=payload)
    #response = response.replace("'", '"')
    #data = json.loads(response)
    data = response.json() # convert json to python-readable format 
    return data


In [205]:
all_data = []
for i in range(1, 11):
    data = get_content(i)
    print data["response"]["status"]
    res = data["response"]["results"]
    all_data += res
#print data["response"]["status"]

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [206]:
print len(all_data)

500


In [207]:
#pprint(res[0])
pprint(all_data[0])

{u'apiUrl': u'http://content.guardianapis.com/us-news/2015/jun/09/carly-fiorina-foreign-policy-republican-interview',
 u'blocks': {u'body': [{u'attributes': {},
                        u'bodyHtml': u'<p>Carly Fiorina has chutzpah. The former CEO of Hewlett Packett isn\u2019t just running for the Republican nomination for president <a href="http://www.theguardian.com/us-news/2015/may/03/carly-fiorina-run-for-president-hewlett-packard">on a platform of managerial expertise</a> after a controversial tenure at that technology company from 1999 to 2005. She\u2019s running as a foreign policy maven despite having never served in elected office.</p> <p>In an interview with the Guardian on Saturday, Fiorina bragged that she \u201cknows more world leaders on the stage\u201d than any of her competitors for the 2016 nomination, a claim she echoed in a speech shortly afterwards at Iowa senator Joni Ernst\u2019s <a href="http://www.theguardian.com/us-news/2015/jun/06/republican-2016-iowa-joni-ernst

In [208]:
if "blocks" in all_data[0] and "fields" in all_data[0]:
    print(all_data[0]["blocks"]["body"][0]["bodyTextSummary"])
    print(all_data[0]["fields"]["byline"])
    #print(all_data[0]["webPublicationDate"][:4])

Carly Fiorina has chutzpah. The former CEO of Hewlett Packett isn’t just running for the Republican nomination for president on a platform of managerial expertise after a controversial tenure at that technology company from 1999 to 2005. She’s running as a foreign policy maven despite having never served in elected office. In an interview with the Guardian on Saturday, Fiorina bragged that she “knows more world leaders on the stage” than any of her competitors for the 2016 nomination, a claim she echoed in a speech shortly afterwards at Iowa senator Joni Ernst’s inaugural Roast and Ride fundraiser. The former CEO argued that she “conducted business and charitable work” with world leaders “for many years”. She contrasted herself with mere politicians by saying: “I don’t do photo ops. I don’t do codels [congressional delegations].” Fiorina insisted “I know these people. I know how they think about us.” Her foreign policy worldview, though, seemed to be more focused on criticism of the Ob

In [209]:
def get_human_names(text):
    if " and " in text or " with " in text:
        #don't really deal with multiple authors
        return text
    elif " in " in text:
        #assume same name in different locations is 
        # still the same person
        ind = text.find(" in ")
        return text[:ind]
    else:
        return text

In [5]:
def dd():
    return ""

In [211]:
def make_dict(loc=False):
    authors = []
    names = []
    auth_ar = defaultdict(dd)
    for b in range(len(all_data)):
        if "fields" in all_data[b]:
            if "blocks" in all_data[b]:
                #print b
                nm = all_data[b]["fields"]["byline"].strip()#.decode("utf-8")
                #print nm
                authors.append(nm)
                if not loc:
                    nm = get_human_names(all_data[b]["fields"]["byline"])
                names.append(nm)
                auth_ar[nm] += all_data[b]["blocks"]["body"][0]["bodyTextSummary"]
    return names, auth_ar
names, auth_ar = make_dict()
print names, "\n", len(names)
#pprint(authors)
counts = Counter(names)
print(counts)

[u'Ben Jacobs', u'Frances Perraudin', u'Ben Jacobs', u'Scott Keyes', u'Michael Cohen', u'Ben Jacobs ', u'Ben Jacobs', u'Paul Lewis', u'Ben Jacobs', u'Jessica Glenza', u'Ben Jacobs in Washington, Sabrina Siddiqui  in Manchester and Jeb Lund', u'Ryan Felton', u'Alan Yuhas with Paul Lewis in Cleveland, Jeb Lund in Tampa, Ben Jacobs in Atlanta, Dan Roberts and Sabrina Siddiqui in Washington and Adam Gabbatt in New York', u'Amanda Holpuch', u'Tom McCarthy', u'Tom Dart', u'Martin Kettle', u'Jeb Lund', u'Paul Lewis', u'Sabrina Siddiqui', u'Observer editorial', u'Dan Roberts', u'Frances Perraudin', u'Paul Lewis', u'Dan Roberts and Sabrina Siddiqui Washington', u'David Boyle', u'Rupert Myers', u'Anna Leach', u'Paul Lewis', u'Ben Jacobs', u'Paul Lewis', u'Jeb Lund', u'Alan Yuhas', u'Scott Lemieux', u'Sabrina Siddiqui', u'Alan Yuhas', u'Paul Lewis', u'Alan Yuhas', u'Patrick Wintour, political editor', u'Frances Perraudin', u'Gary Younge', u'Ben Jacobs in Washington and Alan Yuhas in New York', u'

In [212]:
#politics
print len(auth_ar["Frances Perraudin"])
#sports
#print len(auth_ar["Les Carpenter"])
#scence
#print len(auth_ar["Stuart Clark"])
#arts
#print len(auth_ar["Jonathan Jones"])

38157


In [1]:
#politics
print len(sent_tokenize(auth_ar["Paul Lewis"]))
print sent_tokenize(auth_ar["Paul Lewis"])[0]
#sports
#print len(sent_tokenize(auth_ar["Les Carpenter"]))
#science
#print len(sent_tokenize(auth_ar["Stuart Clark"]))
#arts
#print len(sent_tokenize(auth_ar["Jonathan Jones"]))

NameError: name 'sent_tokenize' is not defined

In [214]:
sents = {}
for author, text in auth_ar.iteritems():
    sent = sent_tokenize(text)
    if len(sent) > 400:
        sents[author] = sent
        print author, len(sent)

Sabrina Siddiqui 582
Paul Lewis 678
Tom McCarthy 576
Dan Roberts 549
Ben Jacobs 675
Jeb Lund 520


In [215]:
with open(topic+"_author_articles.pkl", 'w') as out_auth_ar:
    pickle.dump(auth_ar, out_auth_ar)
with open(topic+"_auth_sents.pkl", 'w') as outs:
    pickle.dump(sents, outs)

In [6]:
with open(topic+"_author_articles.pkl", 'r') as auth_ar_in:
    auth_ar = pickle.load(auth_ar_in)
with open(topic+"_auth_sents.pkl", 'r') as ins:
    sents = pickle.load(ins)
    

In [217]:
def get_frequencies(len_w, counts, subset=False, log=True):
    '''takes number tokens as float,list of (k, v), and converts v from int count to frequency.
    If subset, takes frequency over the counts rather than the whole
    set of tokens'''
    freqs = []
    sorted_counts = zip(*sorted(counts.items()))[1]

    countsum = float(sum(sorted_counts))
    for sc in sorted_counts:
        if subset:
            if countsum > 0.0:
                freqs.append(sc/countsum)
            else:
                freqs.append(0.0)
        else:
            fs = sc/len_w
            if log:
                if fs > 0.0:
                    f = abs(math.log(fs)) #log of fs will always be negative, and mnb can't handle that
                    freqs.append(f)
                else:
                    freqs.append(0.0)
            else:
                freqs.append(fs)
    return freqs

In [218]:
sp = string.punctuation
sp = unicode(sp)
punct = [s for s in sp]

#punct = [u'!', u'$', u'%', u'?', u':', u';', u'``', u"''", u'\u2013']
punct.append(u'``')
punct.append(u"''")
punct.append(u'\u2013')

'''[(u'!', 0), (u'"', 0), (u'#', 0), (u'$', 0), (u'%', 0), (u'&', 0), 
    (u"'", 0), (u"''", 1), (u'(', 0), (u')', 0), (u'*', 0), (u'+', 0), 
    (u',', 1), (u'-', 0), (u'.', 1), (u'/', 0), (u':', 0), (u';', 0), 
    (u'<', 0), (u'=', 0), (u'>', 0), (u'?', 0), (u'@', 0), (u'[', 0), 
    (u'\\', 0), (u']', 0), (u'^', 0), (u'_', 0), (u'`', 0), (u'``', 1), 
    (u'{', 0), (u'|', 0), (u'}', 0), (u'~', 0), (u'\u2013', 0)]'''

#print punct, len(punct)
swords = stopwords.words('english')
swords.append('\'s')
swords.append('\'t')
#print swords

#swear_words = [u'hell', u'shit', u'']
featuresli = []
authors = []
raw_features = []

def pd():
    return 0
label = 1
labels = []
for author, sentences in sents.iteritems():
    print author
    
    for s in sentences:
        #punct_counts = defaultdict(pd)
        punct_counts = dict.fromkeys(punct, 0)
        sword_counts = dict.fromkeys(swords, 0)
        features = []
        len_w = 0.0
        count_w = 0.0
        s = s.replace(u"\u2018", "'").replace(u"\u2019", "'") \
            .replace(u"\u201c",'"').replace(u"\u201d", '"') \
            .replace(u"\u030F", '"').replace(u"''", '"')
        words = word_tokenize(s) #punctuation is included as words
        #print s, "\n", [s]
        #print len(words)
        for word in words:
            word = word.lower()
            #print word, [word]               
            if word in punct:# or word[0] in punct:
                punct_counts[word] += 1
            elif word in swords:
                sword_counts[word] += 1
            len_w += len(word)
            count_w += 1
        avg_wlen = len_w/count_w
        #print avg_wlen
        punct_freq = get_frequencies(len_w, punct_counts)#zip(*sorted(punct_counts.items()))[1]
        sword_freq = get_frequencies(len_w, sword_counts)#zip(*sorted(sword_counts.items()))[1]
        #print punct_freq
        #print sword_freq
        #print sorted(punct_counts.items())
        features.append(len(words)) #total number of tokens in sentence
        features.append(avg_wlen) #average word/token length
        features += punct_freq #frequency of punctuation tokens
        features += sword_freq #frequency of stopword tokens
        rf = [('len_s', len(words)), ('len_w', avg_wlen)]
        rf += sorted(punct_counts.items()) + sorted(sword_counts.items())
        #print rf
        featuresli.append(features)
        authors.append(label)
        raw_features.append(rf)
        
    labels.append((label, author))
    label += 1

print labels
print raw_features[0]
#print authors
#print featuresli
        

Paul Lewis
Dan Roberts
Jeb Lund
Tom McCarthy
Ben Jacobs
Sabrina Siddiqui
[(1, u'Paul Lewis'), (2, u'Dan Roberts'), (3, u'Jeb Lund'), (4, u'Tom McCarthy'), (5, u'Ben Jacobs'), (6, u'Sabrina Siddiqui')]
[('len_s', 21), ('len_w', 4.619047619047619), (u'!', 0), (u'"', 0), (u'#', 0), (u'$', 0), (u'%', 0), (u'&', 0), (u"'", 0), (u"''", 0), (u'(', 0), (u')', 0), (u'*', 0), (u'+', 0), (u',', 2), (u'-', 0), (u'.', 1), (u'/', 0), (u':', 0), (u';', 0), (u'<', 0), (u'=', 0), (u'>', 0), (u'?', 0), (u'@', 0), (u'[', 0), (u'\\', 0), (u']', 0), (u'^', 0), (u'_', 0), (u'`', 0), (u'``', 0), (u'{', 0), (u'|', 0), (u'}', 0), (u'~', 0), (u'\u2013', 0), ("'s", 0), ("'t", 0), (u'a', 0), (u'about', 0), (u'above', 0), (u'after', 0), (u'again', 0), (u'against', 0), (u'all', 0), (u'am', 0), (u'an', 0), (u'and', 0), (u'any', 0), (u'are', 0), (u'as', 0), (u'at', 0), (u'be', 0), (u'because', 0), (u'been', 0), (u'before', 0), (u'being', 0), (u'below', 0), (u'between', 0), (u'both', 0), (u'but', 0), (u'by', 0), (u'ca

In [219]:
with open(topic+"_authors.pkl", 'w') as out_auths:
    pickle.dump(authors, out_auths)
with open(topic+"_features.pkl", 'w') as outs:
    pickle.dump(featuresli, outs)
with open(topic+"_labels.pkl", 'w') as louts:
    pickle.dump(labels, louts)
with open(topic+"_raw_features.pkl", 'w') as rfouts:
    pickle.dump(raw_features, rfouts)